In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [2]:
# O ficheiro datasets/attack_attempts.csv deve ter o formato:
# pokemon_name,attack_type,result

data = np.genfromtxt('datasets/attack_attempts.csv', delimiter=',', dtype=str, skip_header=1)

# Separa colunas
pokemon_names = data[:, 0]
attack_types = data[:, 1]
results = data[:, 2]

## Codificar os dados (Label Encoding)

In [3]:
pokemon_encoder = LabelEncoder()
attack_encoder = LabelEncoder()
result_encoder = LabelEncoder()

pokemon_encoded = pokemon_encoder.fit_transform(pokemon_names)
attack_encoded = attack_encoder.fit_transform(attack_types)
result_encoded = result_encoder.fit_transform(results)

# Junta Pokémon + Tipo de Ataque como input
X = np.column_stack((pokemon_encoded, attack_encoded))
y = result_encoded

## Dividir em treino e teste

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Treinar o modelo Random Forest

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Avaliar no conjunto de teste
accuracy = model.score(X_test, y_test)
print(f"Test accuracy: {accuracy:.2f}")

## Save the trained model and encoders to disk

In [ ]:
import pickle

# Create a dictionary to save everything needed for production
export_dict = {
    "model": model,
    "pokemon_encoder": pokemon_encoder,
    "attack_encoder": attack_encoder,
    "result_encoder": result_encoder
}

# Save it
with open('models/attack_predictor.pkl', 'wb') as f:
    pickle.dump(export_dict, f)

print("Model exported successfully to models/attack_predictor.pkl")

## Exemplo de utilização do modelo treinado

In [10]:
# Carregar o modelo salvo
with open('models/attack_predictor.pkl', 'rb') as f:
    saved_data = pickle.load(f)

# Recuperar o modelo e os codificadores guardados
model = saved_data["model"]
pokemon_encoder = saved_data["pokemon_encoder"]
attack_encoder = saved_data["attack_encoder"]
result_encoder = saved_data["result_encoder"]

In [ ]:
# Suponha que você detectou um Pokémon e os tipos de ataque disponíveis
detected_pokemon_name = "Pidgey"
available_attack_types = ["Electric", "Water", "Fire", "Grass"]

# Para cada ataque disponível, prever o resultado
best_attack = None
best_score = None

for attack in available_attack_types:
    try:
        # Codificar o nome do Pokémon e o tipo de ataque
        pokemon_encoded = pokemon_encoder.transform([detected_pokemon_name])[0]
        attack_encoded = attack_encoder.transform([attack])[0]

        # Fazer a previsão
        prediction = model.predict(np.array([[pokemon_encoded, attack_encoded]]))
        predicted_result = result_encoder.inverse_transform(prediction)[0]

        # Definir um sistema de prioridade
        score = 0
        if predicted_result == "super_effective":
            score = 2
        elif predicted_result == "effective":
            score = 1
        elif predicted_result == "not_very_effective":
            score = 0

        # Atualizar o melhor ataque com base na pontuação
        if best_score is None or score > best_score:
            best_score = score
            best_attack = attack

    except Exception as e:
        print(f"Aviso: não foi possível prever para o ataque {attack} -> {str(e)}")

# Exibir o melhor ataque encontrado
if best_attack:
    print(f"Melhor ataque a usar: {best_attack}")
else:
    print("Nenhum ataque válido encontrado.")